In [40]:
import sys
from ftplib import FTP #import the ftp library
import re 
import os
import pandas as pd
import numpy as np

In [57]:
#t='Dinophyceae'
#t='Bacillariophyta'
#t='Haptophyta'
t='Raphidophyceae'

In [58]:
##retrive files
mt=open('mmetsp_taxonomy.txt','r')
g=[] #make an empty list to store genus names
for line in mt:
    if re.search(t,line): #if taxa name in line
        g= g+line.split('\t')[7:8]#pull out the 8th field should be genus, keeping as list

g=set(g) #keep only unique genus names
#print g
#close the taxonomy file
mt.close()

ftp= FTP('ftp.imicrobe.us') #set home ftp server
ftp.login() #log in
ftp.cwd('camera/combined_assemblies') #ch

files=ftp.nlst() #make a list of all files and directories in wd
delimiter=' '
all=delimiter.join(files)

names=[]

for genus in g:
    string= genus+"\S*.pep.fa.gz"
    taxafiles=re.findall(string, all)
    #print "{} files matching genus=".format(len(taxafiles))+genus
    #print taxafiles
    if len(taxafiles) > 0:
        for filex in taxafiles:
            command = "RETR "+filex
            outfile = filex
            #ftp.retrbinary(command, open(outfile, 'wb').write)
            names.append(outfile)
            
ftp.quit()

if t=='Dinophyceae':
    names.remove('Durinskia-baltica-CSIRO_CS-38.pep.fa.gz')
    names.remove('Oxyrrhis-marina-CCMP1795.pep.fa.gz')
    names.remove('Alexandrium-fundyense-CCMP1719.pep.fa.gz')

print names

['Chattonella-subsalsa-CCMP2191.pep.fa.gz', 'Heterosigma-akashiwo-CCMP2393.pep.fa.gz', 'Heterosigma-akashiwo-CCMP3107.pep.fa.gz', 'Heterosigma-akashiwo-CCMP452.pep.fa.gz', 'Heterosigma-akashiwo-NB.pep.fa.gz']


In [4]:
#names=names[:3]
#print names

['Karenia-brevis-CCMP2229.pep.fa.gz', 'Karenia-brevis-SP1.pep.fa.gz', 'Karenia-brevis-SP3.pep.fa.gz']


In [59]:
def readHMM(Organism,gene_name):
    """ Takes in organism and gene_name from HMM results and makes a table.
    HMM results from --tblout that have the following name organism_genename_HMM.csv
    Note: pep.fa files differ in structure and it can affect how the HMM output is written. If you can't read the file in 
    modify the fuction"""

    hold=pd.read_csv('{}_{}_HMM.csv'.format(Organism,gene_name),delim_whitespace=True,engine='python',skiprows=2,skipfooter=10)
    #empty files have 12 rows so the following if statement will only work on files that are not empty
    
    if hold.shape[0]!=0:
        readX=pd.read_csv('{}_{}_HMM.csv'.format(Organism,gene_name),delim_whitespace=True,engine='python',skiprows=3,
                          header=None ,skipfooter=10)
        
        samplenames=[]
        for i in np.arange(1,readX.shape[1]+1):
            samplenames.append(str(i))

        readX.columns=samplenames
        
        new=pd.DataFrame()
        new['CAMPEPid']=readX['1']
        new['contig']=readX['19']
        new['Evalue']=readX['5']
        new['Annotation']='{}'.format(gene_name)

        new.contig=new.contig.str.split("|").str[1]
        return new
        #return readX

In [60]:
##MAKEHMM
def makeHMM(Organism,gene_type):
    """Combines the plots from each gene_type into a table"""
    if gene_type=='CF':
        a=readHMM(Organism,'PK')
        b=readHMM(Organism,'PEPC')
        c=readHMM(Organism,'PEPCK')
        d=readHMM(Organism,'PK')
        e=readHMM(Organism,'MDH')
        f=readHMM(Organism,'OMT')
        g=readHMM(Organism,'PYC')
        h=readHMM(Organism,'PPDK')
        i=readHMM(Organism,'ME')
        
        frames = [a,b,c,d,e,f,g,i,h]
        result = pd.concat(frames)
        return result
    if gene_type=='PR':
        a=readHMM(Organism,'SHMT')
        b=readHMM(Organism,'GOX')
        c=readHMM(Organism,'GDCT')
        d=readHMM(Organism,'PGP')
        e=readHMM(Organism,'ICL')
        f=readHMM(Organism,'GCL')
        g=readHMM(Organism,'HR')
        h=readHMM(Organism,'SPT')
        i=readHMM(Organism,'TSR')
        j=readHMM(Organism,'MS')
        k=readHMM(Organism,'GlcDH')
        l=readHMM(Organism,'ALAT_GGAT')
        m=readHMM(Organism,'GK')
        
        frames = [a,b,c,d,e,f,g,h,i,j,k,l,m]
        result = pd.concat(frames)
        return result
    if gene_type=='BP':
        a=readHMM(Organism,'CA_alpha')
        b=readHMM(Organism,'CA_delta')
        c=readHMM(Organism,'CA_beta')
        d=readHMM(Organism,'Ca_zeta')
        e=readHMM(Organism,'Bestrophin')
        f=readHMM(Organism,'SLC4')

        frames = [a,b,c,d,e,f]
        result = pd.concat(frames)
        return result

In [61]:
###COUNTS
def HMMcmp(Organism,gene_type):
    result1=makeHMM(Organism,gene_type)
    contig_EV=dict(zip(result1.contig,result1.Evalue))
    contig_Annot=dict(zip(result1.contig,result1.Annotation))
    
    import re
    Organism = re.sub('.pep.fa.gz', '', Organism)
    
    expression=pd.read_csv('{}_cds_counts.txt'.format(Organism),delimiter='\t',index_col=0)
    
    #treatments=len(list(expression))
    #print treatments
    
    expression['log2CPM']=np.log2(expression.sum(axis=1)/1000000)
    expression.index= expression.index.str.split("|").str[1]
    contig_CMP=dict(zip(expression.index,expression.log2CPM))
    
    Contig=[]
    Evalue=[]
    Annotation=[]
    log2CPM=[]
    
    for i in expression.index:
        a=contig_CMP.get(i)
        if a>-16 or a=="-inf" or a=="inf":
            Contig.append(i)
            Evalue.append(contig_EV.get(i))
            Annotation.append(contig_Annot.get(i))
            log2CPM.append(a)
    
    out=pd.DataFrame()
    out['Contig']=Contig
    out['Evalue']=Evalue
    out['Annotation']=Annotation
    out['log2CPM']=log2CPM
        
    return out

In [62]:
HMMcmp('Heterosigma-akashiwo-CCMP2393.pep.fa.gz','BP').head()

,Contig,Evalue,Annotation,log2CPM
0,267033_1,NaN,None,-12.137153
1,3814_1,NaN,None,-9.180025
2,77900_1,NaN,None,-11.266233
3,163778_1,NaN,None,-13.609640
4,156264_1,NaN,None,-15.231129


In [63]:
###CLEAN
def HMMclean(Organism):
    '''Will create an HMM table and filter out values that do not meet an e-value'''
    aX=HMMcmp(Organism,'BP')
    bX=HMMcmp(Organism,'CF')
    cX=HMMcmp(Organism,'PR')
    
    frames=[aX,bX,cX]
    HMM= pd.concat(frames)
     
    HMM=HMM[HMM['Evalue'] < .0001]
    
    #return HMM.drop_duplicates()
    
    Genes= np.unique(HMM['Annotation'])
    AX=pd.DataFrame(0,index=[Organism],columns=Genes)
    
    for j in Genes:
        counts=HMM.Annotation.value_counts()[j]
        AX[j]= counts
           
    return AX
       

In [23]:
HMMclean('Heterosigma-akashiwo-CCMP2393.pep.fa.gz')

,ALAT_GGAT,Bestrophin,CA_alpha,CA_beta,GCL,GDCT,GK,GOX,GlcDH,HR,...,ME,OMT,PEPCK,PGP,PK,PYC,SHMT,SLC4,SPT,TSR
Heterosigma-akashiwo-CCMP2393.pep.fa.gz,13,6,5,9,1,1,1,4,2,5,...,4,62,13,9,3,15,4,3,1,6


In [64]:
###GENE_TYPE
def gen_type(gene_type):
    """ Allows you to extract specific gene_types"""
    
    if gene_type=='BP':
            fixed=['SLC4','Bestrophin','CA_alpha','CA_beta','CA_delta','CA_zeta']
    if gene_type=='PR':
            fixed=['PGP','GOX','SPT','ALAT_GGAT','GDCT','SHMT','HR','GK','GlcDH','MS','ICL','GCL','TSR']
    if gene_type=='CF':
            fixed=['PK','PEPC','PEPCK','MDH','OMT','ME','PPDK','PYC']
            
    A=pd.DataFrame(0, index=names, columns=fixed)
    
    for i,j in enumerate(names):
        for k in fixed:
            B=HMMclean(j)
            #print B
            if k in list(B):
                A[k][i]=B[k]
                
    A.index = A.index.str.split('.').str[0] 
    ####the count is the number of transcriptomes taken into account
    ###I just added gene_name to teh count to make teh values unique for when teh different gene_type frames are concatenated
    A['count{}'.format(gene_type)]=0
    
    for i,j in enumerate(A.index):
        expression=pd.read_csv('{}_cds_counts.txt'.format(j),delimiter='\t',index_col=0)
        A['count{}'.format(gene_type)][i]=len(list(expression))
                
    C=A.transpose()
    C.to_csv('{}_{}_GeneCountHMM.csv'.format(t,gene_type))
    
    return A.transpose()

In [38]:
gen_type('BP')

,Karenia-brevis-CCMP2229,Karenia-brevis-SP1,Karenia-brevis-SP3,Karenia-brevis-Wilson,Peridinium-aciculiferum-PAER_2,Karlodinium-micrum-CCMP2283,Prorocentrum-minimum-CCMP1329,Prorocentrum-minimum-CCMP2233,Symbiodinium-kawagutii-CCMP2468,Symbiodinium-sp-C1,...,Alexandrium-monilatum-CCMP3105,Alexandrium-temarense-CCMP1771,Azadinium-spinosum-3D9,Ceratium-fusus-PA161109,Oxyrrhis-marina-LB1974,Oxyrrhis-marina,Glenodinium-foliaceum-CCAP1116_3,Scrippsiella-hangoei-SHTV5,Scrippsiella-hangoei_like-SHHI_4,Scrippsiella-trochoidea-CCMP3099
SLC4,12,19,14,32,2,3,2,2,1,2,...,4,10,6,5,0,1,2,3,2,0
Bestrophin,132,139,90,128,27,78,49,50,0,32,...,86,85,62,87,54,59,71,61,57,2
CA_alpha,20,19,19,19,25,20,7,6,0,7,...,19,23,19,24,16,17,35,29,30,0
CA_beta,7,6,5,4,18,0,0,0,0,22,...,8,8,7,20,9,15,18,23,28,0
CA_delta,9,7,8,7,0,6,5,7,0,1,...,6,7,2,0,0,0,1,0,0,0
CA_zeta,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
countBP,4,2,2,4,2,3,4,3,4,2,...,4,4,3,2,3,4,2,3,3,3


In [65]:
a=gen_type('BP')
b=gen_type('CF')
c=gen_type('PR')

framesX=(a,b,c)
outFrame=pd.concat(framesX)

outFrame.to_csv('{}_GeneCountHMM.csv'.format(t))

print 'COMPLETE'

COMPLETE
